# 逻辑回归

## what


**假设函数**：$h(x)=\frac{1}{1+e^{-x\cdot\theta}}=p(y=1|x)=p$

参数：$\theta=\{\theta_0,\theta_1,\cdots,\theta_n\}$

最大似然函数：$L(\theta)=\prod^{m}_{i=1}p^{y^{(i)}}(1-p)^{1-y^{(i)}}$

**损失函数**：$J(\theta)=-\frac{1}{m}\sum_{i=1}^{m}[y^{(i)}\ln{p^{(i)}}+(1-y^{(i)})\ln(1-p^{(i)})]$


损失函数链式求导：

$\frac{\partial J}{\partial \theta}=\frac{\partial J}{\partial p}\cdot \frac{\partial p}{\partial z}\cdot \frac{\partial z}{\partial x}=-\frac{1}{m}\sum_{i=1}^{m}\cdot [\frac{y}{p}-\frac{1-y}{1-p}]\cdot [p(1-p)]\cdot x$

**梯度**：$\bigtriangledown J(\theta)=\frac{1}{m}\sum_{i=1}^m(h(x)-y)x=\frac{1}{m}X^{'}(h(\mathbf{x})-\mathbf{y})$


## how

In [1]:
import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix
from sklearn.preprocessing import MaxAbsScaler
from sklearn.decomposition import TruncatedSVD

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import *

### 数据

In [2]:
# 读取csv
train = pd.read_csv('/content/sample_data/mnist_train_small.csv')
test = pd.read_csv('/content/sample_data/mnist_test.csv')

# 二分类样本
train_bin_cat = train[train.iloc[:, 0].isin([0, 1])]
test_bin_cat = test[test.iloc[:, 0].isin([0, 1])]

# 标签、特征
X_train, y_train = train_bin_cat.iloc[:, 1:], train_bin_cat.iloc[:, 0]
X_test, y_test = test_bin_cat.iloc[:, 1:], test_bin_cat.iloc[:, 0]

train_bin_cat.head()

,6,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.10,0.11,0.12,0.13,0.14,0.15,0.16,0.17,0.18,0.19,0.20,0.21,0.22,0.23,0.24,0.25,0.26,0.27,0.28,0.29,0.30,0.31,0.32,0.33,0.34,0.35,0.36,0.37,0.38,...,0.551,0.552,0.553,0.554,0.555,0.556,0.557,0.558,0.559,0.560,0.561,0.562,0.563,0.564,0.565,0.566,0.567,0.568,0.569,0.570,0.571,0.572,0.573,0.574,0.575,0.576,0.577,0.578,0.579,0.580,0.581,0.582,0.583,0.584,0.585,0.586,0.587,0.588,0.589,0.590
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
34,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
41,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
45,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [3]:
# 稀疏矩阵
X_train = csr_matrix(X_train)
X_test = csr_matrix(X_test)

# 标准化
scaler = MaxAbsScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 降维
dec = TruncatedSVD(n_components=96)
X_train = dec.fit_transform(X_train)
X_test = dec.transform(X_test)
print(dec.explained_variance_ratio_.sum())
print(X_test.shape)

0.9503513883517918
(2115, 96)


### 模型

In [4]:
clf = LogisticRegression(solver='saga', max_iter=500, random_state=100)
params = {
    'C': [0.001, 0.01, 0.1, 1, 10]
}
grid = GridSearchCV(clf, params, cv=5, scoring='roc_auc', verbose=2, n_jobs=-1)
grid.fit(X_train, y_train)
print(grid.best_params_)
print(grid.best_score_)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:   24.7s finished


{'C': 0.1}
0.9999931835737745


In [5]:
model = grid.best_estimator_
y_hat = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]
print(f'accuracy：{accuracy_score(y_test, y_hat)}')
print(f'precision：{precision_score(y_test, y_hat)}')
print(f'recall：{recall_score(y_test, y_hat)}')
print(f'f1：{f1_score(y_test, y_hat)}')
print(f'auc：{roc_auc_score(y_test, y_prob)}')
print(confusion_matrix(y_test, y_hat))
print(classification_report(y_test, y_hat))

accuracy：0.9990543735224586
precision：0.9982409850483729
recall：1.0
f1：0.9991197183098591
auc：1.0
[[ 978    2]
 [   0 1135]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       980
           1       1.00      1.00      1.00      1135

    accuracy                           1.00      2115
   macro avg       1.00      1.00      1.00      2115
weighted avg       1.00      1.00      1.00      2115



## why

In [6]:
class MyLogReg():
    def __init__(self, eta=1.0, thread=1e-4, max_iter=100):
        self.X = None
        self.y = None
        self.theta = None  # 参数
        self._eta = eta  # 学习率
        self._thread = thread  # 阈值
        self._max_iter = max_iter  # 最大迭代次数
    
    def _sigmoid(self, z):  # sigmoid函数
        return 1 / (1 + np.exp(-z))
    
    def _entropy(self, h):  # 交叉熵
        return -self.y*np.log(h) - (1-self.y)*np.log(1-h)
    
    def _J(self):  # 损失函数
        h = self._sigmoid(self.X @ self.theta)
        m = len(self.y)
        try:
            return np.sum(self._entropy(h)) / m
        except:
            return float('inf')
    
    def _dJ(self):  # 梯度
        h = self._sigmoid(self.X @ self.theta)
        m = len(self.y)
        return (self.X.T @ (h - self.y)) / m
    
    def fit(self, X_train, y_train):
        self.X = np.c_[np.ones(X_train.shape[0]), X_train]
        self.y = np.array(y_train).reshape(-1,1)
        self.theta = np.zeros(self.X.shape[1]).reshape(-1,1)
        count = 0
        while count < self._max_iter:
            old_J = self._J()
            dJ = self._dJ()
            self.theta -= self._eta * dJ
            new_J = self._J()
            if np.abs(new_J - old_J) < self._thread:
                break
            count +=1
        return self

    def predict_proba(self, X_test):
        X_b = np.c_[np.ones(X_test.shape[0]), X_test]
        return np.array(X_b @ self.theta).reshape(-1,1)
    
    def predict(self, X_test):
        X_b = np.c_[np.ones(X_test.shape[0]), X_test]
        return np.array(X_b@self.theta>0, dtype=np.int64)
    
    def score(self, X_test, y_test):
        y_pred = self.predict(X_test).reshape(-1,)
        y_test = np.array(y_test).reshape(-1,)
        return np.sum(y_pred==y_test) / len(y_test)

In [7]:
myModel = MyLogReg(eta=0.1)
myModel.fit(X_train, y_train)
myModel.score(X_test, y_test)

0.9990543735224586